In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import openpathsampling as paths
import openpathsampling.storage as st

In [ ]:
%%time
storage = st.AnalysisStorage("mstis.nc")

In [ ]:
len(storage.steps)

In [ ]:
mstis = storage.networks.load(0)
retis = mstis.sampling_transitions[0]
scheme = storage.schemes[0]

### Defining labels

Everything is actually done in terms of ensembles. We can map the ensembles to any labels. In our case, we use the initial replica ID associated with the ensemble. We use this as both the numeric and string label.

In [ ]:
sset0 = storage.samplesets[0]
numeric_labels = {s.ensemble : s.replica for s in sset0}
string_labels = {s.ensemble : str(s.replica) for s in sset0}
numeric_to_string = {numeric_labels[e] : string_labels[e] for e in numeric_labels.keys()}

## Trace of ensemble visited by a replica

In the plot below, you'll see we set the labels on the axis as sset0[e].replica, so we use the replica ID associated with the ensemble in the first timestep.

In [ ]:
%%time
trace_1 = paths.trace_ensembles_for_replica(0, storage.steps)

In [ ]:
plt.plot([numeric_labels[e] for e in trace_1])

### Replica flow

In [ ]:
%%time
repx_net = paths.ReplicaNetwork(scheme, storage.steps)

In [ ]:
flow = repx_net.flow(bottom=retis.minus_ensemble, top=retis.ensembles[-1],
                     included_ensembles=[retis.minus_ensemble]+retis.ensembles)
print(flow)

In [ ]:
flow_num = {numeric_labels[k] : flow[k] for k in flow.keys()}
print(flow_num)

In [ ]:
sorted_vals = []
for k in sorted(flow_num.keys()):
    sorted_vals.append(flow_num[k])

In [ ]:
plt.plot(sorted(flow_num.keys()), sorted_vals)

### Trips

Now we calculate "up" trips, "down" trips, and round-trips.

In [ ]:
repx_net.trips(bottom=retis.minus_ensemble, top=retis.ensembles[-1])

### Transition matrix

The transition matrix is the (unsymmetrized) matrix of the transition probabilities. By default, it automatically sets its order from the Cuthill-McKee reverse ordering algorithm.

In [ ]:
repx_net.transition_matrix().style
# .style has the effect that 0.0 is reported as 0 instead of 0.000000

If you would like to set a different order, that can be done by providing a list of the ensembles in whatever order you choose:

In [ ]:
import numpy as np
perm = np.random.permutation(len(mstis.all_ensembles))
print(perm)

In [ ]:
order = [mstis.all_ensembles[p] for p in perm]
repx_net.transition_matrix(index_order=order).style

### Mixing matrix

Same as the transition matrix $T$, but $\frac{1}{2}(T+T^T)$.

In [ ]:
repx_net.mixing_matrix().style

### Making a pretty picture

In [ ]:
repxG = paths.ReplicaNetworkGraph(repx_net)

In [ ]:
# draw('graphviz') gives better results, but requires pygraphviz
repxG.draw('spring')

Blue is a minus interface, red is a normal interface. Multiple state outer interfaces (not in this example) would be green.